In [1]:
from joblib import Parallel, delayed
from tqdm import tqdm

import numpy as np
import polars as pl
from lrv_test import LRV
from scipy.linalg import sqrtm

import pickle
from pathlib import Path
from typing import Callable

from typing import Optional
from data_generation.arma import generate_arma_sample, generate_ar_covariance_matrix
from data_generation.spectral_density import _arma_spectral_density

from data_generation.other import factor_model_sample, common_random_dependence_sample

# Default values

In [23]:
# for the LRV test, we will always use the following test functions
f = lambda x: (x - 1) ** 2

# we will always evaluate the test against these values of N and M
# the results are valid for alpha < 7/9 which is approx 0.7778.
# alpha=2/3 should give the best results
alpha = 0.66
N_range = np.linspace(1000, 8000, 8)
c = 1 / 2
gamma = 1 / 4  # L = int(N**gamma)

# default values for the iid time series. We choose a small AR as that
# creates a very spiky spectral density if too large
time_ar, time_ma = 0.1, 0.5


def get_freqs(N: int, B: int) -> np.ndarray:
    # frequencies to evaluate, exclude -0.5 and 0 as they
    # have not well behaved: big estimation error at these point
    # for spectral density
    return np.arange(-0.4, 0.6, B / N)


# always repeat an experiment K times
n_repeats = 10**4

# threshold to detect
level = 0.1

# generate real gaussian data
is_complex_gaussian = True

# various computation (complex contour integral) should result in a
# real value. We check that the imaginary part is smaller than this
tolerance = 1e-5

# path to the stored experiment results for easy reloading
data_storage_path = "./data/{experiment_type}/{experiment_id}.pickle"

# Batch tools

In [24]:
# Define the functions to store and load experiment results
def run_batch(
    run: Callable, n_repeats: int, experiment_type: str, run_kwargs: dict
) -> None:
    c = run_kwargs["M"] / run_kwargs["B"]

    from lrv_test.contour import Contour
    from lrv_test.functions import support_MP, t
    from lrv_test.sigma import compute_sigma
    from lrv_test.utils import action_D_on_f, contour_integral, psi

    f_against_mp = action_D_on_f(f, lambda z: t(z, c), support_MP(c), tolerance)
    support = (-np.sqrt(c), np.sqrt(c))
    radius = (support[1] - support[0]) / 2
    center = (support[0] + support[1]) / 2
    contour = Contour.from_circle_parameters(center, radius)
    f_against_D = contour_integral(
        lambda w: -c / (2 * np.pi * 1j) * f(psi(w, c)) / (w**3), contour
    )
    f_against_D = np.real(f_against_D)
    sigma = compute_sigma(f, c, tolerance)
    precomputed = (f_against_mp, f_against_D, sigma)

    experiment_id = "_".join(
        f"{key}={value}"
        for key, value in run_kwargs.items()
        if isinstance(value, int) or isinstance(value, float) or isinstance(value, str)
    )
    resolved_data_path = Path(
        data_storage_path.format(
            experiment_type=experiment_type, experiment_id=experiment_id
        )
    )
    if resolved_data_path.exists():
        return
        # pass
    print(f"Running {experiment_id}")

    resolved_data_path.parent.mkdir(parents=True, exist_ok=True)

    results = [run(precomputed, **run_kwargs) for _ in range(n_repeats)]
    with open(resolved_data_path, "wb") as handle:
        pickle.dump(results, handle)


def load_batch(experiment_type: str) -> pl.DataFrame:
    # load the samples
    all_results = []

    resolved_data_path = Path(
        data_storage_path.format(experiment_type=experiment_type, experiment_id="none")
    ).parent
    for file in resolved_data_path.iterdir():
        if file.stem == ".DS_Store":
            continue

        with open(file, "rb") as handle:
            results = pickle.load(handle)
        all_results += results

    df = pl.DataFrame(all_results)
    return df

# Level check

In [25]:
def run(
    precomputed,
    N,
    M,
    B,
    L,
    c,
    time_ar,
    time_ma,
    is_complex_gaussian,
    coef_type: Optional[str] = None,
    oracle_sd: Optional[Callable] = None,
):
    f_against_mp, f_against_D, sigma = precomputed
    y = generate_arma_sample(
        N,
        M,
        time_ar,
        time_ma,
        is_complex_gaussian,
    )

    freqs = get_freqs(N, B)
    lrv_result = LRV(
        y,
        B,
        f,
        freqs,
        L,
        sd=oracle_sd,
        f_against_mp=f_against_mp,
        f_against_D=f_against_D,
        sigma=sigma,
    )

    return {
        "N": N,
        "M": M,
        "B": B,
        "L": L,
        "c": c,
        # "time_ar": time_ar,
        # "time_ma": time_ma,
        "is_complex_gaussian": is_complex_gaussian,
        "t_stat_1": lrv_result.t_stat_1,
        "t_stat_2": lrv_result.t_stat_2,
        "t_stat_3": lrv_result.t_stat_3,
        "t_stat_4": lrv_result.t_stat_4,
        "is_positive_1": lrv_result.is_positive_1(level),
        "is_positive_2": lrv_result.is_positive_2(level),
        "is_positive_3": lrv_result.is_positive_3(level),
        "is_positive_4": lrv_result.is_positive_4(level),
        "coef_type": coef_type,
    }

### Constant vs Random ARMA parameter

In [5]:
tasks = []
for coef_type in ["constant", "random"]:
    for N in N_range:
        L = int(N**gamma)
        B = int(N**alpha)
        if B % 2 == 0:
            B -= 1
        M = int(B * c)

        if coef_type == "constant":
            ar, ma = time_ar, time_ma
        else:
            ar, ma = np.random.uniform(-0.5, 0.5, M), np.random.uniform(-0.5, 0.5, M)

        task = (
            run,
            n_repeats,
            f"arma_no_dependence",
            {
                "N": int(N),
                "M": int(M),
                "B": int(B),
                "L": int(L),
                "c": c,
                "time_ar": ar,
                "time_ma": ma,
                "is_complex_gaussian": is_complex_gaussian,
                "coef_type": coef_type,
            },
        )

        tasks.append(task)

# shuffle the tasks to distribute the load over time
np.random.shuffle(tasks)
results = Parallel(n_jobs=4)(delayed(run_batch)(*task) for task in tqdm(tasks))

df = load_batch(f"arma_no_dependence")
df = df.unpivot(
    index=["N", "M", "B", "L", "c", "is_complex_gaussian", "coef_type"],
    on=[f"is_positive_{i}" for i in range(1, 5)],
    value_name="is_positive",
    variable_name="test",
)
pivot = (
    df.group_by(["N", "test", "coef_type"])
    .mean()
    .sort(by=["N", "coef_type", "test"])
    .to_pandas()
    .pivot(index="N", columns=["coef_type", "test"], values="is_positive")
)
print(pivot.to_latex(float_format="%.3f"))

  0%|          | 0/16 [00:00<?, ?it/s]

Running N=6000_M=155_B=311_L=8_c=0.5_time_ar=0.1_time_ma=0.5_is_complex_gaussian=True_coef_type=constant
Running N=3000_M=98_B=197_L=7_c=0.5_time_ar=0.1_time_ma=0.5_is_complex_gaussian=True_coef_type=constant
Running N=4000_M=118_B=237_L=7_c=0.5_is_complex_gaussian=True_coef_type=random
Running N=2000_M=74_B=149_L=6_c=0.5_is_complex_gaussian=True_coef_type=random


 50%|█████     | 8/16 [00:02<00:02,  3.03it/s]

Running N=4000_M=118_B=237_L=7_c=0.5_time_ar=0.1_time_ma=0.5_is_complex_gaussian=True_coef_type=constant
Running N=1000_M=47_B=95_L=5_c=0.5_time_ar=0.1_time_ma=0.5_is_complex_gaussian=True_coef_type=constant
Running N=5000_M=137_B=275_L=8_c=0.5_is_complex_gaussian=True_coef_type=random
Running N=8000_M=187_B=375_L=9_c=0.5_is_complex_gaussian=True_coef_type=random


 75%|███████▌  | 12/16 [00:05<00:01,  2.12it/s]

Running N=1000_M=47_B=95_L=5_c=0.5_is_complex_gaussian=True_coef_type=random
Running N=3000_M=98_B=197_L=7_c=0.5_is_complex_gaussian=True_coef_type=random
Running N=7000_M=171_B=343_L=9_c=0.5_is_complex_gaussian=True_coef_type=random


100%|██████████| 16/16 [00:07<00:00,  2.14it/s]

Running N=5000_M=137_B=275_L=8_c=0.5_time_ar=0.1_time_ma=0.5_is_complex_gaussian=True_coef_type=constant
Running N=2000_M=74_B=149_L=6_c=0.5_time_ar=0.1_time_ma=0.5_is_complex_gaussian=True_coef_type=constant


Running N=6000_M=155_B=311_L=8_c=0.5_is_complex_gaussian=True_coef_type=random
Running N=8000_M=187_B=375_L=9_c=0.5_time_ar=0.1_time_ma=0.5_is_complex_gaussian=True_coef_type=constant
Running N=7000_M=171_B=343_L=9_c=0.5_time_ar=0.1_time_ma=0.5_is_complex_gaussian=True_coef_type=constant
\begin{tabular}{lrrrrrrrr}
\toprule
coef_type & \multicolumn{4}{r}{constant} & \multicolumn{4}{r}{random} \\
test & is_positive_1 & is_positive_2 & is_positive_3 & is_positive_4 & is_positive_1 & is_positive_2 & is_positive_3 & is_positive_4 \\
N &  &  &  &  &  &  &  &  \\
\midrule
1000 & 0.300 & 0.100 & 0.200 & 0.100 & 0.000 & 0.100 & 0.100 & 0.100 \\
2000 & 0.000 & 0.200 & 0.100 & 0.100 & 0.100 & 0.000 & 0.000 & 0.100 \\
3000 & 0.200 & 0.200 & 0.300 & 0.100 & 0.100 & 0.100 & 0.000 & 0.000 \\
4000 & 0.000 & 0.000 & 0.100 & 0.100 & 0.300 & 0.000 & 0.000 & 0.000 \\
5000 & 0.000 & 0.000 & 0.000 & 0.100 & 0.300 & 0.200 & 0.100 & 0.100 \\
6000 & 0.100 & 0.100 & 0.000 & 0.000 & 0.000 & 0.100 & 0.000 & 0.000

In [6]:
pivot

coef_type      constant                                            \
test      is_positive_1 is_positive_2 is_positive_3 is_positive_4   
N                                                                   
1000                0.3           0.1           0.2           0.1   
2000                0.0           0.2           0.1           0.1   
3000                0.2           0.2           0.3           0.1   
4000                0.0           0.0           0.1           0.1   
5000                0.0           0.0           0.0           0.1   
6000                0.1           0.1           0.0           0.0   
7000                0.1           0.0           0.0           0.2   
8000                0.2           0.2           0.1           0.2   

coef_type        random                                            
test      is_positive_1 is_positive_2 is_positive_3 is_positive_4  
N                                                                  
1000                0.0           0.1           0.1           0.1  
2000                0.1           0.0           0.0           0.1  
3000                0.1           0.1           0.0           0.0  
4000                0.3           0.0           0.0           0.0  
5000                0.3           0.2           0.1           0.1  
6000                0.0           0.1           0.0           0.0  
7000                0.1           0.1           0.1           0.1  
8000                0.1           0.3           0.0           0.0

### AR grid

In [7]:
ma = 0
ar_range = np.arange(0.1, 0.8, 0.2)

tasks = []
for N in N_range:
    for ar in ar_range:
        L = int(N**gamma)
        B = int(N**alpha)
        if B % 2 == 0:
            B -= 1
        M = int(B * c)

        task = (
            run,
            n_repeats,
            f"arma_no_dependence_multiple_ar",
            {
                "N": int(N),
                "M": int(M),
                "B": int(B),
                "L": int(L),
                "c": c,
                "time_ar": ar,
                "time_ma": ma,
                "is_complex_gaussian": is_complex_gaussian,
            },
        )

        tasks.append(task)

        # also generate the oracle version
        oracle_sd = _arma_spectral_density(ar, ma)

        task = (
            run,
            n_repeats,
            f"arma_no_dependence_multiple_ar_oracle",
            {
                "N": int(N),
                "M": int(M),
                "B": int(B),
                "L": int(L),
                "c": c,
                "time_ar": ar,
                "time_ma": ma,
                "oracle_sd": oracle_sd,
                "is_complex_gaussian": is_complex_gaussian,
            },
        )
        tasks.append(task)

# shuffle the tasks to distribute the load over time
np.random.shuffle(tasks)
results = Parallel(n_jobs=4)(delayed(run_batch)(*task) for task in tqdm(tasks))

  0%|          | 0/64 [00:00<?, ?it/s]

Running N=4000_M=118_B=237_L=7_c=0.5_time_ar=0.5000000000000001_time_ma=0_is_complex_gaussian=True
Running N=4000_M=118_B=237_L=7_c=0.5_time_ar=0.1_time_ma=0_is_complex_gaussian=True
Running N=7000_M=171_B=343_L=9_c=0.5_time_ar=0.5000000000000001_time_ma=0_is_complex_gaussian=True
Running N=5000_M=137_B=275_L=8_c=0.5_time_ar=0.1_time_ma=0_is_complex_gaussian=True


 12%|█▎        | 8/64 [00:03<00:24,  2.32it/s]

Running N=1000_M=47_B=95_L=5_c=0.5_time_ar=0.7000000000000002_time_ma=0_is_complex_gaussian=True
Running N=3000_M=98_B=197_L=7_c=0.5_time_ar=0.30000000000000004_time_ma=0_is_complex_gaussian=True
Running N=2000_M=74_B=149_L=6_c=0.5_time_ar=0.30000000000000004_time_ma=0_is_complex_gaussian=True
Running N=1000_M=47_B=95_L=5_c=0.5_time_ar=0.5000000000000001_time_ma=0_is_complex_gaussian=True


 19%|█▉        | 12/64 [00:05<00:26,  1.97it/s]

...

Running N=2000_M=74_B=149_L=6_c=0.5_time_ar=0.5000000000000001_time_ma=0_is_complex_gaussian=True
Running N=7000_M=171_B=343_L=9_c=0.5_time_ar=0.30000000000000004_time_ma=0_is_complex_gaussian=True
Running N=4000_M=118_B=237_L=7_c=0.5_time_ar=0.7000000000000002_time_ma=0_is_complex_gaussian=True
Running N=3000_M=98_B=197_L=7_c=0.5_time_ar=0.30000000000000004_time_ma=0_is_complex_gaussian=True


 94%|█████████▍| 60/64 [01:06<00:05,  1.43s/it]

Running N=4000_M=118_B=237_L=7_c=0.5_time_ar=0.30000000000000004_time_ma=0_is_complex_gaussian=True
Running N=6000_M=155_B=311_L=8_c=0.5_time_ar=0.7000000000000002_time_ma=0_is_complex_gaussian=True
Running N=1000_M=47_B=95_L=5_c=0.5_time_ar=0.30000000000000004_time_ma=0_is_complex_gaussian=True
Running N=2000_M=74_B=149_L=6_c=0.5_time_ar=0.1_time_ma=0_is_complex_gaussian=True


100%|██████████| 64/64 [01:07<00:00,  1.06s/it]


Running N=7000_M=171_B=343_L=9_c=0.5_time_ar=0.7000000000000002_time_ma=0_is_complex_gaussian=True
Running N=2000_M=74_B=149_L=6_c=0.5_time_ar=0.30000000000000004_time_ma=0_is_complex_gaussian=True
Running N=8000_M=187_B=375_L=9_c=0.5_time_ar=0.30000000000000004_time_ma=0_is_complex_gaussian=True
Running N=7000_M=171_B=343_L=9_c=0.5_time_ar=0.7000000000000002_time_ma=0_is_complex_gaussian=True


In [9]:
df = load_batch(f"arma_no_dependence_multiple_ar")
df = df.melt(
    id_vars=["N", "M", "B", "L", "c", "is_complex_gaussian"],
    value_vars=[f"is_positive_{i}" for i in range(1, 5)],
    value_name="is_positive",
    variable_name="test",
)
df = df.group_by(["N", "M", "B", "L", "c", "is_complex_gaussian", "test"]).mean()

/var/folders/ng/g1qy_rr97098d2b5gwfq8_s40000gn/T/ipykernel_62318/1242391768.py:2: DeprecationWarning: `DataFrame.melt` is deprecated. Use `unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  df = df.melt(


In [11]:
df_oracle = load_batch(f"arma_no_dependence_multiple_ar_oracle")
df_oracle = df_oracle.melt(
    id_vars=["N", "M", "B", "L", "c", "is_complex_gaussian"],
    value_vars=[f"is_positive_{i}" for i in range(1, 5)],
    value_name="is_oracle_positive",
    variable_name="test",
)
df_oracle = df_oracle.group_by(
    ["N", "M", "B", "L", "c", "is_complex_gaussian", "test"]
).mean()

/var/folders/ng/g1qy_rr97098d2b5gwfq8_s40000gn/T/ipykernel_62318/2761182397.py:2: DeprecationWarning: `DataFrame.melt` is deprecated. Use `unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  df_oracle = df_oracle.melt(


In [14]:
# merge df and df_oracle on N, M, B, L, c, time_ar, time_ma, is_complex_gaussian, ζ
df = df.join(
    df_oracle,
    on=["N", "M", "B", "L", "c", "is_complex_gaussian", "test"],
)

In [13]:
pivot = (
    df.filter(pl.col("test").is_in(["is_positive_3"]))
    .to_pandas()
    .pivot(
        index="N",
        columns=["time_ar"],
        values=["is_positive", "is_oracle_positive"],
    )
)
print(pivot.to_latex(float_format="%.3f"))

KeyError: 'time_ar'

In [ ]:
pivot

is_positive                  is_oracle_positive                  
time_ar         0.1   0.3   0.5  0.7                0.1   0.3   0.5   0.7
N                                                                        
1000           0.10  0.07  0.18  1.0               0.06  0.08  0.07  0.31
2000           0.06  0.05  0.17  1.0               0.03  0.10  0.10  0.44
3000           0.09  0.12  0.07  1.0               0.12  0.12  0.04  0.21
4000           0.09  0.10  0.09  1.0               0.07  0.06  0.10  0.20
5000           0.09  0.08  0.14  1.0               0.14  0.16  0.08  0.20
6000           0.07  0.11  0.10  1.0               0.11  0.11  0.15  0.19
7000           0.11  0.06  0.14  1.0               0.09  0.10  0.10  0.17
8000           0.12  0.07  0.10  1.0               0.06  0.10  0.07  0.19

### Correction

In [26]:
alpha = 0.6

tasks = []
for N in N_range:
    L = int(N**gamma)
    B = int(N**alpha)
    if B % 2 == 0:
        B -= 1
    M = int(B * c)

    # compute the corrective term in s_m(nu)
    oracle_sd = None

    task = (
        run,
        n_repeats,
        f"arma_no_dependence_with_correction",
        {
            "N": int(N),
            "M": int(M),
            "B": int(B),
            "L": int(L),
            "c": c,
            "time_ar": time_ar,
            "time_ma": time_ma,
            "oracle_sd": oracle_sd,
            "is_complex_gaussian": is_complex_gaussian,
        },
    )
    tasks.append(task)

    # don't compute the corrective term in s_m(nu)
    oracle_sd = None
    L = None

    task = (
        run,
        n_repeats,
        f"arma_no_dependence_no_correction",
        {
            "N": int(N),
            "M": int(M),
            "B": int(B),
            "L": None,
            "c": c,
            "time_ar": time_ar,
            "time_ma": time_ma,
            "oracle_sd": oracle_sd,
            "is_complex_gaussian": is_complex_gaussian,
        },
    )
    tasks.append(task)

# shuffle the tasks to distribute the load over time
np.random.shuffle(tasks)
results = Parallel(n_jobs=4)(delayed(run_batch)(*task) for task in tqdm(tasks))

  0%|          | 0/16 [00:00<?, ?it/s]

Running N=6000_M=91_B=183_L=8_c=0.5_time_ar=0.1_time_ma=0.5_is_complex_gaussian=True
Running N=8000_M=109_B=219_c=0.5_time_ar=0.1_time_ma=0.5_is_complex_gaussian=True
Running N=7000_M=100_B=201_L=9_c=0.5_time_ar=0.1_time_ma=0.5_is_complex_gaussian=True
Running N=2000_M=47_B=95_c=0.5_time_ar=0.1_time_ma=0.5_is_complex_gaussian=True


 50%|█████     | 8/16 [04:55<04:55, 36.97s/it]

Running N=8000_M=109_B=219_L=9_c=0.5_time_ar=0.1_time_ma=0.5_is_complex_gaussian=True
Running N=1000_M=31_B=63_c=0.5_time_ar=0.1_time_ma=0.5_is_complex_gaussian=True
Running N=3000_M=60_B=121_c=0.5_time_ar=0.1_time_ma=0.5_is_complex_gaussian=True
Running N=6000_M=91_B=183_c=0.5_time_ar=0.1_time_ma=0.5_is_complex_gaussian=True


 75%|███████▌  | 12/16 [1:03:23<25:48, 387.00s/it]

Running N=4000_M=71_B=143_c=0.5_time_ar=0.1_time_ma=0.5_is_complex_gaussian=True
Running N=3000_M=60_B=121_L=7_c=0.5_time_ar=0.1_time_ma=0.5_is_complex_gaussian=True
Running N=5000_M=82_B=165_c=0.5_time_ar=0.1_time_ma=0.5_is_complex_gaussian=True
Running N=4000_M=71_B=143_L=7_c=0.5_time_ar=0.1_time_ma=0.5_is_complex_gaussian=True


100%|██████████| 16/16 [1:24:03<00:00, 315.22s/it]


Running N=2000_M=47_B=95_L=6_c=0.5_time_ar=0.1_time_ma=0.5_is_complex_gaussian=True
Running N=5000_M=82_B=165_L=8_c=0.5_time_ar=0.1_time_ma=0.5_is_complex_gaussian=True
Running N=1000_M=31_B=63_L=5_c=0.5_time_ar=0.1_time_ma=0.5_is_complex_gaussian=True
Running N=7000_M=100_B=201_c=0.5_time_ar=0.1_time_ma=0.5_is_complex_gaussian=True


In [27]:
df

N,M,B,c,is_complex_gaussian,test,is_positive,is_positive_no_rn
i64,i64,i64,f64,bool,str,f64,f64
8000,109,219,0.5,true,"""is_positive_4""",0.0,0.0
3000,60,121,0.5,true,"""is_positive_4""",0.0,0.1
6000,91,183,0.5,true,"""is_positive_1""",0.1,0.2
5000,82,165,0.5,true,"""is_positive_1""",0.1,0.5
7000,100,201,0.5,true,"""is_positive_4""",0.1,0.1
…,…,…,…,…,…,…,…
5000,82,165,0.5,true,"""is_positive_2""",0.1,0.0
2000,47,95,0.5,true,"""is_positive_2""",0.2,0.2
4000,71,143,0.5,true,"""is_positive_3""",0.0,0.1


In [28]:
df = load_batch(f"arma_no_dependence_with_correction").drop("L")
df = df.melt(
    id_vars=["N", "M", "B", "c", "is_complex_gaussian"],
    value_vars=[f"is_positive_{i}" for i in range(1, 5)],
    value_name="is_positive",
    variable_name="test",
)
df = df.group_by(["N", "M", "B", "c", "is_complex_gaussian", "test"]).mean()

/var/folders/ng/g1qy_rr97098d2b5gwfq8_s40000gn/T/ipykernel_62318/1270341570.py:2: DeprecationWarning: `DataFrame.melt` is deprecated. Use `unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  df = df.melt(


In [29]:
df_no = load_batch(f"arma_no_dependence_no_correction").drop("L")
df_no = df_no.melt(
    id_vars=["N", "M", "B", "c", "is_complex_gaussian"],
    value_vars=[f"is_positive_{i}" for i in range(1, 5)],
    value_name="is_positive_no_rn",
    variable_name="test",
)
df_no = df_no.group_by(["N", "M", "B", "c", "is_complex_gaussian", "test"]).mean()

/var/folders/ng/g1qy_rr97098d2b5gwfq8_s40000gn/T/ipykernel_62318/1893697820.py:2: DeprecationWarning: `DataFrame.melt` is deprecated. Use `unpivot` instead, with `index` instead of `id_vars` and `on` instead of `value_vars`
  df_no = df_no.melt(


In [30]:
# merge df and df_oracle on N, M, B, L, c, time_ar, time_ma, is_complex_gaussian, ζ
df = df.join(
    df_no,
    on=["N", "M", "B", "c", "is_complex_gaussian", "test"],
)

In [31]:
pivot = (
    # df.filter(pl.col("test").is_in(["test_3", "test_oracle_3"]))
    # .with_columns(
    #     [
    #         (pl.col("p_value") < level).alias("is_test_positive").cast(pl.Float64),
    #         (pl.col("p_value_oracle") < level)
    #         .alias("is_oracle_test_positive")
    #         .cast(pl.Float64),
    #     ]
    # )
    # .group_by(["N", "time_ar", "time_ma", "test"])
    # .mean()
    # df.filter(~pl.col('time_ar').is_in([0.0, 0.2, 0.4, 0.6]))
    df.to_pandas().pivot(
        index="N",
        columns=["test"],
        values=["is_positive", "is_positive_no_rn"],
    )
)
print(pivot.to_latex(float_format="%.3f"))

\begin{tabular}{lrrrrrrrr}
\toprule
 & \multicolumn{4}{r}{is_positive} & \multicolumn{4}{r}{is_positive_no_rn} \\
test & is_positive_1 & is_positive_2 & is_positive_3 & is_positive_4 & is_positive_1 & is_positive_2 & is_positive_3 & is_positive_4 \\
N &  &  &  &  &  &  &  &  \\
\midrule
1000 & 0.098 & 0.079 & 0.089 & 0.072 & 0.244 & 0.120 & 0.146 & 0.119 \\
2000 & 0.115 & 0.096 & 0.097 & 0.071 & 0.278 & 0.127 & 0.150 & 0.113 \\
3000 & 0.107 & 0.087 & 0.093 & 0.077 & 0.254 & 0.110 & 0.133 & 0.108 \\
4000 & 0.099 & 0.082 & 0.086 & 0.073 & 0.247 & 0.106 & 0.129 & 0.103 \\
5000 & 0.100 & 0.091 & 0.091 & 0.070 & 0.269 & 0.113 & 0.138 & 0.105 \\
6000 & 0.102 & 0.092 & 0.090 & 0.081 & 0.253 & 0.103 & 0.128 & 0.097 \\
7000 & 0.100 & 0.091 & 0.090 & 0.074 & 0.254 & 0.105 & 0.127 & 0.099 \\
8000 & 0.104 & 0.090 & 0.095 & 0.081 & 0.270 & 0.113 & 0.134 & 0.103 \\
\bottomrule
\end{tabular}



In [32]:
pivot

is_positive                                            \
test is_positive_1 is_positive_2 is_positive_3 is_positive_4   
N                                                              
1000        0.0982        0.0789        0.0890        0.0719   
2000        0.1153        0.0961        0.0965        0.0714   
3000        0.1068        0.0872        0.0926        0.0772   
4000        0.0991        0.0820        0.0863        0.0730   
5000        0.0999        0.0913        0.0906        0.0701   
6000        0.1020        0.0919        0.0905        0.0805   
7000        0.1003        0.0909        0.0901        0.0739   
8000        0.1036        0.0897        0.0946        0.0807   

     is_positive_no_rn                                            
test     is_positive_1 is_positive_2 is_positive_3 is_positive_4  
N                                                                 
1000            0.2439        0.1197        0.1462        0.1186  
2000            0.2782        0.1269        0.1499        0.1126  
3000            0.2542        0.1100        0.1334        0.1082  
4000            0.2473        0.1064        0.1294        0.1035  
5000            0.2689        0.1131        0.1380        0.1051  
6000            0.2525        0.1032        0.1276        0.0974  
7000            0.2537        0.1051        0.1271        0.0989  
8000            0.2704        0.1130        0.1336        0.1031

# Power check

## AR covariance structure

In [ ]:
def run(
    precomputed,
    N,
    M,
    B,
    L,
    c,
    time_ar,
    time_ma,
    space_ar,
    is_complex_gaussian,
):
    f_against_mp, f_against_D, sigma = precomputed
    y = generate_arma_sample(
        N,
        M,
        time_ar,
        time_ma,
        is_complex_gaussian,
    )
    Cov = generate_ar_covariance_matrix(M, space_ar)
    y = y @ sqrtm(Cov)

    freqs = get_freqs(N, B)

    lrv_result = LRV(
        y,
        B,
        f,
        freqs,
        L,
        f_against_mp=f_against_mp,
        f_against_D=f_against_D,
        sigma=sigma,
    )
    return {
        "N": N,
        "M": M,
        "B": B,
        "L": L,
        "c": c,
        "time_ar": time_ar,
        "time_ma": time_ma,
        "space_ar": space_ar,
        "is_complex_gaussian": is_complex_gaussian,
        "t_stat_1": lrv_result.t_stat_1,
        "t_stat_2": lrv_result.t_stat_2,
        "t_stat_3": lrv_result.t_stat_3,
        "t_stat_4": lrv_result.t_stat_4,
        "is_positive_1": lrv_result.is_positive_1(level),
        "is_positive_2": lrv_result.is_positive_2(level),
        "is_positive_3": lrv_result.is_positive_3(level),
        "is_positive_4": lrv_result.is_positive_4(level),
    }

In [ ]:
# ARMA covariance structure
space_ar = 0.05

tasks = []
for N in N_range:
    L = int(N**gamma)
    B = int(N**alpha)
    if B % 2 == 0:
        B -= 1
    M = int(B * c)

    task = (
        run,
        n_repeats,
        f"ar_dependence",
        {
            "N": int(N),
            "M": int(M),
            "B": int(B),
            "L": int(L),
            "c": c,
            "time_ar": time_ar,
            "time_ma": time_ma,
            "space_ar": space_ar,
            "is_complex_gaussian": is_complex_gaussian,
        },
    )

    tasks.append(task)

# shuffle the tasks to distribute the load over time
np.random.shuffle(tasks)
results = Parallel(n_jobs=4)(delayed(run_batch)(*task) for task in tqdm(tasks))

df = load_batch(f"ar_dependence").drop(["time_ar", "time_ma"])
df = df.unpivot(
    index=["N", "M", "B", "L", "c", "is_complex_gaussian"],
    on=[f"is_positive_{i}" for i in range(1, 5)],
    value_name="is_positive",
    variable_name="test",
)

  0%|          | 0/8 [00:00<?, ?it/s]

Running N=7000_M=171_B=343_L=9_c=0.5_time_ar=0.1_time_ma=0.5_space_ar=0.05_is_complex_gaussian=True
Running N=2000_M=74_B=149_L=6_c=0.5_time_ar=0.1_time_ma=0.5_space_ar=0.05_is_complex_gaussian=True
Running N=8000_M=187_B=375_L=9_c=0.5_time_ar=0.1_time_ma=0.5_space_ar=0.05_is_complex_gaussian=True
Running N=1000_M=47_B=95_L=5_c=0.5_time_ar=0.1_time_ma=0.5_space_ar=0.05_is_complex_gaussian=True


100%|██████████| 8/8 [00:02<00:00,  2.72it/s]


Running N=3000_M=98_B=197_L=7_c=0.5_time_ar=0.1_time_ma=0.5_space_ar=0.05_is_complex_gaussian=True
Running N=4000_M=118_B=237_L=7_c=0.5_time_ar=0.1_time_ma=0.5_space_ar=0.05_is_complex_gaussian=True
Running N=6000_M=155_B=311_L=8_c=0.5_time_ar=0.1_time_ma=0.5_space_ar=0.05_is_complex_gaussian=True
Running N=5000_M=137_B=275_L=8_c=0.5_time_ar=0.1_time_ma=0.5_space_ar=0.05_is_complex_gaussian=True


In [ ]:
pivot = (
    df.group_by(["N", "test"])
    .mean()
    .to_pandas()
    .pivot(index="N", columns=["test"], values="is_positive")
)
print(pivot.to_latex(float_format="%.3f"))

\begin{tabular}{lrrrr}
\toprule
test & is_positive_1 & is_positive_2 & is_positive_3 & is_positive_4 \\
N &  &  &  &  \\
\midrule
1000 & 0.180 & 0.110 & 0.170 & 0.090 \\
2000 & 0.500 & 0.230 & 0.290 & 0.160 \\
3000 & 0.790 & 0.400 & 0.460 & 0.250 \\
4000 & 0.960 & 0.610 & 0.670 & 0.400 \\
5000 & 0.990 & 0.800 & 0.840 & 0.460 \\
6000 & 1.000 & 0.870 & 0.890 & 0.570 \\
7000 & 1.000 & 0.960 & 0.960 & 0.650 \\
8000 & 1.000 & 0.970 & 0.980 & 0.750 \\
\bottomrule
\end{tabular}



In [ ]:
pivot

test,is_positive_1,is_positive_2,is_positive_3,is_positive_4
N,,,,
1000,0.18,0.11,0.17,0.09
2000,0.50,0.23,0.29,0.16
3000,0.79,0.40,0.46,0.25
4000,0.96,0.61,0.67,0.40
5000,0.99,0.80,0.84,0.46
6000,1.00,0.87,0.89,0.57
7000,1.00,0.96,0.96,0.65
8000,1.00,0.97,0.98,0.75


## Common random dependence


In [ ]:
def run(
    precomputed,
    N,
    M,
    B,
    L,
    c,
    time_ar,
    time_ma,
    mixing_scale,
    is_complex_gaussian,
):
    f_against_mp, f_against_D, sigma = precomputed

    freqs = get_freqs(N, B)

    y = common_random_dependence_sample(
        N, M, time_ar, time_ma, mixing_scale, is_complex_gaussian
    )

    lrv_result = LRV(
        y,
        B,
        f,
        freqs,
        L,
        f_against_mp=f_against_mp,
        f_against_D=f_against_D,
        sigma=sigma,
    )
    return {
        "N": N,
        "M": M,
        "B": B,
        "L": L,
        "c": c,
        "time_ar": time_ar,
        "time_ma": time_ma,
        "mixing_scale": mixing_scale,
        "is_complex_gaussian": is_complex_gaussian,
        "t_stat_1": lrv_result.t_stat_1,
        "t_stat_2": lrv_result.t_stat_2,
        "t_stat_3": lrv_result.t_stat_3,
        "t_stat_4": lrv_result.t_stat_4,
        "is_positive_1": lrv_result.is_positive_1(level),
        "is_positive_2": lrv_result.is_positive_2(level),
        "is_positive_3": lrv_result.is_positive_3(level),
        "is_positive_4": lrv_result.is_positive_4(level),
    }

In [ ]:
mixing_scale = 0.05

tasks = []
for N in N_range:
    L = int(N**gamma)
    B = int(N**alpha)
    if B % 2 == 0:
        B -= 1
    M = int(B * c)

    task = (
        run,
        n_repeats,
        "common_random_dependence",
        {
            "N": int(N),
            "M": int(M),
            "B": int(B),
            "L": int(L),
            "c": c,
            "time_ar": time_ar,
            "time_ma": time_ma,
            "mixing_scale": mixing_scale,
            "is_complex_gaussian": is_complex_gaussian,
        },
    )

    tasks.append(task)

# shuffle the tasks to distribute the load over time
np.random.shuffle(tasks)
results = Parallel(n_jobs=4)(delayed(run_batch)(*task) for task in tqdm(tasks))

df = load_batch(f"common_random_dependence").drop(["time_ar", "time_ma"])
df = df.unpivot(
    index=["N", "M", "B", "L", "c", "is_complex_gaussian"],
    on=[f"is_positive_{i}" for i in range(1, 5)],
    value_name="is_positive",
    variable_name="test",
)

  0%|          | 0/8 [00:00<?, ?it/s]

Running N=1000_M=47_B=95_L=5_c=0.5_time_ar=0.1_time_ma=0.5_mixing_scale=0.05_is_complex_gaussian=True
Running N=8000_M=187_B=375_L=9_c=0.5_time_ar=0.1_time_ma=0.5_mixing_scale=0.05_is_complex_gaussian=True
Running N=3000_M=98_B=197_L=7_c=0.5_time_ar=0.1_time_ma=0.5_mixing_scale=0.05_is_complex_gaussian=True
Running N=2000_M=74_B=149_L=6_c=0.5_time_ar=0.1_time_ma=0.5_mixing_scale=0.05_is_complex_gaussian=True


100%|██████████| 8/8 [00:02<00:00,  2.99it/s]


Running N=6000_M=155_B=311_L=8_c=0.5_time_ar=0.1_time_ma=0.5_mixing_scale=0.05_is_complex_gaussian=True
Running N=4000_M=118_B=237_L=7_c=0.5_time_ar=0.1_time_ma=0.5_mixing_scale=0.05_is_complex_gaussian=True
Running N=5000_M=137_B=275_L=8_c=0.5_time_ar=0.1_time_ma=0.5_mixing_scale=0.05_is_complex_gaussian=True
Running N=7000_M=171_B=343_L=9_c=0.5_time_ar=0.1_time_ma=0.5_mixing_scale=0.05_is_complex_gaussian=True


In [ ]:
pivot = (
    df.group_by(["N", "test"])
    .mean()
    .to_pandas()
    .pivot(index="N", columns=["test"], values="is_positive")
)
print(pivot.to_latex(float_format="%.3f"))

\begin{tabular}{lrrrr}
\toprule
test & is_positive_1 & is_positive_2 & is_positive_3 & is_positive_4 \\
N &  &  &  &  \\
\midrule
1000 & 0.160 & 0.220 & 0.240 & 0.150 \\
2000 & 0.470 & 0.270 & 0.290 & 0.220 \\
3000 & 0.740 & 0.380 & 0.470 & 0.300 \\
4000 & 0.890 & 0.530 & 0.560 & 0.360 \\
5000 & 0.990 & 0.630 & 0.690 & 0.390 \\
6000 & 1.000 & 0.880 & 0.910 & 0.560 \\
7000 & 1.000 & 0.920 & 0.950 & 0.640 \\
8000 & 1.000 & 0.990 & 1.000 & 0.780 \\
\bottomrule
\end{tabular}



In [ ]:
pivot

test,is_positive_1,is_positive_2,is_positive_3,is_positive_4
N,,,,
1000,0.16,0.22,0.24,0.15
2000,0.47,0.27,0.29,0.22
3000,0.74,0.38,0.47,0.30
4000,0.89,0.53,0.56,0.36
5000,0.99,0.63,0.69,0.39
6000,1.00,0.88,0.91,0.56
7000,1.00,0.92,0.95,0.64
8000,1.00,0.99,1.00,0.78


## Factor model 

In [ ]:
def run(
    precomputed,
    N,
    M,
    B,
    L,
    r,
    snr,
    time_ar,
    time_ma,
    is_complex_gaussian,
):
    f_against_mp, f_against_D, sigma = precomputed

    freqs = get_freqs(N, B)

    y = factor_model_sample(N, M, r, time_ar, time_ma, snr, is_complex_gaussian)

    lrv_result = LRV(
        y,
        B,
        f,
        freqs,
        L,
        f_against_mp=f_against_mp,
        f_against_D=f_against_D,
        sigma=sigma,
    )
    return {
        "N": N,
        "M": M,
        "B": B,
        "L": L,
        "r": r,
        "c": c,
        "snr": snr,
        "time_ar": time_ar,
        "time_ma": time_ma,
        "is_complex_gaussian": is_complex_gaussian,
        "t_stat_1": lrv_result.t_stat_1,
        "t_stat_2": lrv_result.t_stat_2,
        "t_stat_3": lrv_result.t_stat_3,
        "t_stat_4": lrv_result.t_stat_4,
        "is_positive_1": lrv_result.is_positive_1(level),
        "is_positive_2": lrv_result.is_positive_2(level),
        "is_positive_3": lrv_result.is_positive_3(level),
        "is_positive_4": lrv_result.is_positive_4(level),
    }

In [ ]:
snr_range = [0.05, 0.1]
r = 1

tasks = []
for snr in snr_range:
    for N in N_range:
        L = int(N**gamma)
        B = int(N**alpha)
        if B % 2 == 0:
            B -= 1
        M = int(B * c)

        task = (
            run,
            n_repeats,
            f"factor_model",
            {
                "N": int(N),
                "M": int(M),
                "B": int(B),
                "L": int(L),
                "r": r,
                "snr": snr,
                "time_ar": time_ar,
                "time_ma": time_ma,
                "is_complex_gaussian": is_complex_gaussian,
            },
        )

        tasks.append(task)

# shuffle the tasks to distribute the load over time
np.random.shuffle(tasks)
results = Parallel(n_jobs=4)(delayed(run_batch)(*task) for task in tqdm(tasks))

df = load_batch(f"factor_model")
df = df.unpivot(
    index=["N", "M", "B", "L", "c", "snr", "r", "is_complex_gaussian"],
    on=[f"is_positive_{i}" for i in range(1, 5)],
    value_name="is_positive",
    variable_name="test",
)

  0%|          | 0/16 [00:00<?, ?it/s]

Running N=6000_M=155_B=311_L=8_r=1_snr=0.1_time_ar=0.1_time_ma=0.5_is_complex_gaussian=True
Running N=3000_M=98_B=197_L=7_r=1_snr=0.05_time_ar=0.1_time_ma=0.5_is_complex_gaussian=True
Running N=2000_M=74_B=149_L=6_r=1_snr=0.1_time_ar=0.1_time_ma=0.5_is_complex_gaussian=True
Running N=4000_M=118_B=237_L=7_r=1_snr=0.05_time_ar=0.1_time_ma=0.5_is_complex_gaussian=True


 50%|█████     | 8/16 [00:11<00:11,  1.38s/it]

Running N=1000_M=47_B=95_L=5_r=1_snr=0.05_time_ar=0.1_time_ma=0.5_is_complex_gaussian=True
Running N=8000_M=187_B=375_L=9_r=1_snr=0.1_time_ar=0.1_time_ma=0.5_is_complex_gaussian=True
Running N=2000_M=74_B=149_L=6_r=1_snr=0.05_time_ar=0.1_time_ma=0.5_is_complex_gaussian=True
Running N=1000_M=47_B=95_L=5_r=1_snr=0.1_time_ar=0.1_time_ma=0.5_is_complex_gaussian=True


 75%|███████▌  | 12/16 [00:28<00:10,  2.67s/it]

Running N=7000_M=171_B=343_L=9_r=1_snr=0.05_time_ar=0.1_time_ma=0.5_is_complex_gaussian=True
Running N=8000_M=187_B=375_L=9_r=1_snr=0.05_time_ar=0.1_time_ma=0.5_is_complex_gaussian=True
Running N=4000_M=118_B=237_L=7_r=1_snr=0.1_time_ar=0.1_time_ma=0.5_is_complex_gaussian=True
Running N=3000_M=98_B=197_L=7_r=1_snr=0.1_time_ar=0.1_time_ma=0.5_is_complex_gaussian=True


100%|██████████| 16/16 [01:29<00:00,  5.62s/it]


Running N=6000_M=155_B=311_L=8_r=1_snr=0.05_time_ar=0.1_time_ma=0.5_is_complex_gaussian=True
Running N=5000_M=137_B=275_L=8_r=1_snr=0.1_time_ar=0.1_time_ma=0.5_is_complex_gaussian=True
Running N=5000_M=137_B=275_L=8_r=1_snr=0.05_time_ar=0.1_time_ma=0.5_is_complex_gaussian=True
Running N=7000_M=171_B=343_L=9_r=1_snr=0.1_time_ar=0.1_time_ma=0.5_is_complex_gaussian=True


In [ ]:
pivot = (
    df.group_by(["N", "test", "snr"])
    .mean()
    .to_pandas()
    .pivot(index="N", columns=["snr", "test"], values="is_positive")
)
pivot = pivot.sort_index(axis=1, level=[0, 1])
print(pivot.to_latex(float_format="%.3f"))

\begin{tabular}{lrrrrrrrr}
\toprule
snr & \multicolumn{4}{r}{0.050000} & \multicolumn{4}{r}{0.100000} \\
test & is_positive_1 & is_positive_2 & is_positive_3 & is_positive_4 & is_positive_1 & is_positive_2 & is_positive_3 & is_positive_4 \\
N &  &  &  &  &  &  &  &  \\
\midrule
1000 & 0.110 & 0.100 & 0.100 & 0.080 & 0.140 & 0.100 & 0.100 & 0.120 \\
2000 & 0.150 & 0.110 & 0.130 & 0.080 & 0.670 & 0.510 & 0.520 & 0.450 \\
3000 & 0.110 & 0.110 & 0.090 & 0.080 & 0.920 & 0.870 & 0.890 & 0.870 \\
4000 & 0.150 & 0.080 & 0.090 & 0.070 & 1.000 & 0.990 & 0.990 & 0.980 \\
5000 & 0.150 & 0.080 & 0.130 & 0.070 & 1.000 & 1.000 & 1.000 & 1.000 \\
6000 & 0.200 & 0.140 & 0.170 & 0.060 & 1.000 & 1.000 & 1.000 & 1.000 \\
7000 & 0.260 & 0.170 & 0.200 & 0.110 & 1.000 & 1.000 & 1.000 & 1.000 \\
8000 & 0.380 & 0.130 & 0.180 & 0.170 & 1.000 & 1.000 & 1.000 & 1.000 \\
\bottomrule
\end{tabular}



In [ ]:
import numpy as np

10 * np.log10(0.05), 10 * np.log10(0.10)

(np.float64(-13.010299956639813), np.float64(-10.0))